NOTE: each query takes a very long time on each pdf on google colab. ~10 minutes per pdf.

In [ ]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers
!pip install spacy
!pip install pdfminer.six

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import PyPDFLoader
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


Import Mistral7B Model using LLamaCpp. Can adjust parameters (temperature, top_p). llmloc is for location extraction


In [ ]:
#Import Model
llmloc = LlamaCpp(
    streaming = False,
    model_path="/content/drive/MyDrive/WWF x DSS/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.01,
    top_p=0.01,
    verbose=True,
    n_ctx=5000
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


Another import of Mistral 7B model but with parameters slightly changed for better summary extraction

In [ ]:
llmsummary = LlamaCpp(
    streaming = False,
    model_path="/content/drive/MyDrive/WWF x DSS/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=5000
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


Import huggingfaceembeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

Given file path to a folder full of pdfs, this cell converts into text and puts into arrays.

In [ ]:
pdf_directory = '/content/drive/MyDrive/WWF x DSS/19 good pdf'
list_of_texts = []
name_of_texts = []
for file_name in os.listdir(pdf_directory):
  full_path = os.path.join(pdf_directory, file_name)
  loader = PyPDFLoader(full_path)
  data = loader.load()
  list_of_texts.append(data)
  name_of_texts.append(file_name)

Extracts locations using spacy only

In [ ]:
import spacy
from pdfminer.high_level import extract_text
# Load SpaCy's NER model
nlp = spacy.load("en_core_web_sm")
# Function to extract locations using SpaCy
def extract_geolocations(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ == "GPE"]
# Folder path is the wwf pdf names folder path and pdf_names is a list of the pdfs you want to do this for
# List to hold the dictionary for creating DataFrame later
data_for_df = []
# Iterate over PDF filenames to extract locations
for pdf_name in name_of_texts:
    pdf_path = os.path.join(pdf_directory, pdf_name)
    # Extract text from PDF
    text = extract_text(pdf_path)
    # Extract locations from text
    locations = extract_geolocations(text)
    # Append the results to our list
    data_for_df.append({"PDF Name": pdf_name, "Locations": locations})
# Create a DataFrame
df = pd.DataFrame(data_for_df)
# Show the DataFrame
df

,PDF Name,Locations
0,4bd9978c-41b5-4a4d-bc4b-0a50a15c9d8d.pdf,"[Nepal, Mongolia, Pakistan, Mongolia, Pakistan..."
1,7adc790c-cde9-4581-80cc-4b196a1d5dfa.pdf,"[US, BMDS, spills6, Oregon, WASHINGTON, US, Or..."
2,1646f364-e360-42d8-9c0a-0a84f63e0265.pdf,"[Mexico, Belize, Guatemala, Honduras, China, G..."
3,057385c7-73d0-4ffe-8a0e-02b6521e781d.pdf,"[Honduras, Mexico, Belize, Guatemala, Honduras..."
4,ea2b521c-7a6e-42d0-935b-ad7265b30a28.pdf,"[COMMUNITIES, US, KYRGYZ, Pakistan, UK, the Un..."
5,feca6b34-bee5-46df-a34e-dbf33dd672a1.pdf,"[US, Bering, Chukchi, Washington, DC, US]"
6,f65582c6-adbb-4de7-9096-4df08064916d.pdf,"[Bhutan, India, Mongolia, Nepal, Pakistan, Nep..."
7,76d11bb3-328b-4e51-a56c-dbbc52832f14.pdf,"[COMMUNITIES, the United States \n\nAgency for..."
8,3a5b9550-0d83-49b2-b2af-1a55e27f9e69.pdf,"[US, UK, US, US, California, Hawai’i, Virginia..."
9,5a35c033-666e-4dc8-aacf-d6ba9fa37d8f.pdf,"[the Ivory Ban, China, China, ivory, pangolin,..."


setting up retrievalQA for both locations and summary querys.

In [ ]:
#locations query
text_splitter1 = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
text_chunks1 = text_splitter1.split_documents(list_of_texts[19])
vector_store1 = FAISS.from_documents(text_chunks1, embedding=embeddings)
qa1 = RetrievalQA.from_chain_type(llm=llmloc, chain_type="stuff", retriever=vector_store1.as_retriever(search_kwargs={"k": 2}))

#summary query
text_splitter2 = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=20)
text_chunks2 = text_splitter2.split_documents(list_of_texts[19])
vector_store2 = FAISS.from_documents(text_chunks2, embedding=embeddings)
qa2 = RetrievalQA.from_chain_type(llm=llmsummary, chain_type="stuff", retriever=vector_store2.as_retriever(search_kwargs={"k": 2}))

IndexError: ignored

the questions we are asking LLM to answer.

In [ ]:
query_summary = "Can you give me a general summary of the text?"
query_locstest = f"According to the text, which of these locations ({list_of_locs}) are geographic locations where world wildlife fund did work in?"

Run the location query. Note: might need to rerun if empty output and may take 5-10 minutes

In [ ]:
qa1.run(query_locstest)

Llama.generate: prefix-match hit


' The World Wildlife Fund (WWF) worked in the following locations according to the text: Mongolia, Nepal, Kyrgyzstan, and WWF-Mongolia.'

run the summary query. Note: might need to rerun if empty output and may take 5-10 minutes


In [ ]:
qa2.run(query_summary)

Llama.generate: prefix-match hit


KeyboardInterrupt: ignored

should manually run each pdf one at a time and store outputs into below lists


In [ ]:
pdfs = []
locations = []
summaries = []

Steps below convert lists into dataframe and then downloads as csv

In [ ]:
data = []
for i in range(len(summaries)):
    data.append([pdfs[i], locations[i], summaries[i]])


In [ ]:
locations_df = pd.DataFrame(data, columns=["PDF Name", "Locations", "Summary"])
locations_df

,PDF Name,Locations,Summary
0,4bd9978c-41b5-4a4d-bc4b-0a50a15c9d8d.pdf,The World Wildlife Fund (WWF) has worked in t...,"to summarize the text, GSLEP and the governmen..."
1,7adc790c-cde9-4581-80cc-4b196a1d5dfa.pdf,"According to the text, World Wildlife Fund (W...",The text describes a school food program in wh...
2,1646f364-e360-42d8-9c0a-0a84f63e0265.pdf,"The World Wildlife Fund worked in Mexico, Bel...","In this passage, WWF and Coca-Cola are partne..."
3,057385c7-73d0-4ffe-8a0e-02b6521e781d.pdf,The World Wildlife Fund (WWF) has worked in t...,The text describes a partnership between Coca...
4,ea2b521c-7a6e-42d0-935b-ad7265b30a28.pdf,The World Wildlife Fund (WWF) has worked in t...,The World Wildlife Fund (WWF) is an internati...
5,feca6b34-bee5-46df-a34e-dbf33dd672a1.pdf,The locations where World Wildlife Fund did w...,The Arctic is an extraordinary place facing u...
6,f65582c6-adbb-4de7-9096-4df08064916d.pdf,"According to the text, the World Wildlife Fun...",The text describes an ongoing project called ...
7,76d11bb3-328b-4e51-a56c-dbbc52832f14.pdf,The World Wildlife Fund (WWF) has worked in s...,This article is about a project called Conser...
8,3a5b9550-0d83-49b2-b2af-1a55e27f9e69.pdf,The text does not mention any of the location...,The text discusses the importance of measurin...
9,5a35c033-666e-4dc8-aacf-d6ba9fa37d8f.pdf,The text does not mention any specific locati...,The text presents research on the effectivene...


In [ ]:
locations_df.to_csv('mistral_results.csv')